# Subsetting, Anomalies, and Climatologies

In a previous notebook (`Reanalysis`) we created a relatively large database of raw $U,V,\psi$.
This is helpful, but for our visualizations it's nice to sub-set the data in advance so that all we have to do is load the relevant bits.
We'll also pre-compute anomalies and climatologies, which saves substantial computational time down the road at the expense of a bit of file storage.
Because we don't need the fine time resolution, we'll also re-sample everything to a daily time step (note that we didn't just download daily reanalysis data because we need to shift the end-of-day time!)

In [1]:
import xarray as xr
import glob
from paraguayfloodspy.xrutil import *
from paraguayfloodspy.pars import *

Get some parameters

In [2]:
pars = GetPars('all')
pars

{'clim_plot': {'latmax': 10, 'latmin': -55, 'lonmax': 360, 'lonmin': 240},
 'months': [11, 12, 1, 2],
 'rain': {'latmax': 10, 'latmin': -55, 'lonmax': 330, 'lonmin': 270},
 'rpy_rain': {'latmax': -22.75,
  'latmin': -26.75,
  'lonmax': 304.75,
  'lonmin': 301.25},
 'time': {'eyear': 2016, 'syear': 1979},
 'wt_rgn': {'latmax': -15, 'latmin': -30, 'lonmax': 315, 'lonmin': 295}}

## Reanalysis Data

Now define a transfer function.
This function needs to:
1. Apply a time shift to the data to correct for the end-of-day difference between reanalysis and the CPC Rain data (see our paper for details)
2. Re-sample data to **daily time step**
3. Sub-set data to include only the desired months

In [3]:
def transfer_fn(var, lonmin, lonmax, latmin, latmax, shift_time_h, return_daily, months):
    var = var.sel(lon = slice(lonmin, lonmax), lat = slice(latmax, latmin))
    # shift time by hours, if desired
    if shift_time_h != 0:
        time_old = var.time.values
        time_new = time_old + np.timedelta64(12, 'h')
        var['time'] = time_new
    # resample to daily time step, if desired
    if return_daily:
        var = var.resample('1D', dim = 'time')
    # subset months if required
    if months is not None:
        var = var.sel(time = np.in1d(var['time.month'], months))
    return(var)

`read_netcdfs` needs this as a `lambda` function

In [4]:
trans_lam = lambda ds: transfer_fn(
    var=ds, 
    lonmin=pars['clim_plot']['lonmin'], 
    lonmax=pars['clim_plot']['lonmax'], 
    latmin=pars['clim_plot']['latmin'],
    latmax=pars['clim_plot']['latmax'],
    months=pars['months'],
    shift_time_h=12, return_daily=True)

Now we can go through and read in the data.

In [5]:
for var in ['uwnd', 'vwnd', 'streamfunc']:
    for level_i in [200, 850]:
        fnames = glob.glob('../_data/reanalysis/raw/{}_{}_*.nc'.format(var, level_i))
        if len(fnames) > 0:
            ds = read_netcdfs(fnames, dim='time', transform_func=trans_lam)
            climatology, anomaly = CalcAnomaly(ds, ret_clim=True)
            ds.to_netcdf("../_data/reanalysis/subset/{}_{}_raw.nc".format(var, level_i))
            anomaly.to_netcdf("../_data/reanalysis/subset/{}_{}_anom.nc".format(var, level_i))
            climatology.to_netcdf("../_data/reanalysis/subset/{}_{}_clim.nc".format(var, level_i))
        else:
            print("No {} data at {}hPa".format(var, level_i))

## Rainfall Data

N.B: the rainfall data latiutde data is arranged slightly differently!

In [6]:
def transfer_fn(var, lonmin, lonmax, latmin, latmax, months):
    var = var.sel(lon = slice(lonmin, lonmax), lat = slice(latmin, latmax))
    if months is not None:
        var = var.sel(time = np.in1d(var['time.month'], months))
    return(var)

In [7]:
trans_lam = lambda ds: transfer_fn(
    var=ds, 
    lonmin=pars['rain']['lonmin'], 
    lonmax=pars['rain']['lonmax'], 
    latmin=pars['rain']['latmin'],
    latmax=pars['rain']['latmax'],
    months=pars['months']
)

In [8]:
fnames = glob.glob('../_data/rainfall/raw/*.nc')
if len(fnames) > 0:
    ds = read_netcdfs(fnames, dim='time', transform_func=trans_lam)
    climatology, anomaly = CalcAnomaly(ds, ret_clim=True)
    ds.to_netcdf("../_data/rainfall/subset/cpc_raw.nc")
    anomaly.to_netcdf("../_data/rainfall/subset/cpc_anom.nc")
    climatology.to_netcdf("../_data/rainfall/subset/cpc_clim.nc")
else:
    print("No rainfall data")